In [1]:
import sys
sys.path.append('..')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from interface.load_data import (
    load_data_fault_free_test,
    load_data_fault_free_train,
    load_data_faulty_test,
    load_data_faulty_train,
    load_data_fault_free_test_select

)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
from keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping

In [3]:
faulty_train = load_data_faulty_train()
faulty_train.shape

(5000000, 55)

In [4]:
mask = [i*10 for i in range(int(faulty_train.shape[0]/10))]

faulty_train_w = faulty_train.iloc[mask]
faulty_train_w.shape

(500000, 55)

In [ ]:
#list = [slice() for i in range]
#np.r_list

In [5]:
list_slice = [slice(i * 50, (i+1) * 50) for i in range(1000)]
#list_fault = [[list_slice[0]...list_slice[49], []]
list_fault = [[list_slice[i+j*50] for i in range(50)] for j in range(20)]

In [ ]:
def choose_faults(len_sample=500):
    list_slice = [slice(i * len_sample, (i+1) * len_sample) for i in range(len_sample*20)]
    list_fault = [[list_slice[i+j*len_sample] for i in range(50)] for j in range(20)]


In [9]:
X = faulty_train.drop(columns=['faultNumber', 'simulationRun', 'sample'])
y = faulty_train.faultNumber

In [6]:
X_w = faulty_train_w.drop(columns=['faultNumber', 'simulationRun', 'sample'])
y_w = faulty_train_w.faultNumber

In [53]:
X_feat_select = faulty_train_w[['xmeas_6', 'xmeas_18', 'xmeas_7', 'xmeas_13']]
y_feat_select = faulty_train_w.faultNumber

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_w, y_w, test_size=0.3)

In [10]:
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

In [11]:
n_simulation = 500
len_simulation = 10000
n_per_sample = 50
n_fault = 20
n_features = 52

In [12]:
X_train_reshaped = X_train_scaled.values.reshape(round(len_simulation*0.7), n_per_sample, n_features)
y_train_reshaped = y_train.values.reshape(round(len_simulation*0.7), n_per_sample)[:, 0]

X_test_reshaped = X_test_scaled.values.reshape(round(len_simulation*0.3), n_per_sample, n_features)
y_test_reshaped = y_test.values.reshape(round(len_simulation*0.3), n_per_sample)[:, 0]

In [13]:
print(X_train_reshaped.shape)
print(y_train_reshaped.shape)

(7000, 50, 52)
(7000,)


In [25]:
model = Sequential([
    Input(shape=(50, 52)),

    layers.Conv1D(64, kernel_size=7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(128, kernel_size=5, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(512, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Flatten(),

    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),

    layers.Dense(21, activation='softmax')
])

In [29]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
callbacks = [EarlyStopping(patience=15)]

history = model.fit(
    X_train_reshaped, y_train_reshaped,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.7200 - loss: 1.1444
Epoch 2/100
  8/219 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8009 - loss: 0.7340

/Users/titouan/.pyenv/versions/3.10.6/envs/monitor-reactor/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.7706 - loss: 0.8242
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.7984 - loss: 0.7209
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8046 - loss: 0.6771
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8307 - loss: 0.6023
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8344 - loss: 0.5573
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8523 - loss: 0.4994
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8641 - loss: 0.4552
Epoch 9/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8676 - loss: 0.4504
Epoch 10/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8884 - loss: 0.3904
Epoch 11/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8947 - loss: 0.3534
Epoch 12/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8837 - loss: 0.4002
Epoch 13/100
219/219 ━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [31]:
model.evaluate(X_test_reshaped, y_test_reshaped)

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2297 - loss: 4.5222


[4.522157669067383, 0.22966666519641876]

In [48]:
y_pred = model.predict(X_test_reshaped)
y_pred_pross = np.argmax(y_pred, axis=1)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [50]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_reshaped, y_pred_pross))

              precision    recall  f1-score   support

           1       0.70      0.84      0.76       159
           2       0.82      0.88      0.85       155
           3       0.08      0.06      0.07       166
           4       0.07      0.06      0.06       142
           5       0.08      0.09      0.08       149
           6       0.91      0.89      0.90       141
           7       0.19      0.12      0.15       160
           8       0.14      0.12      0.13       145
           9       0.04      0.05      0.05       146
          10       0.09      0.05      0.06       159
          11       0.07      0.10      0.08       142
          12       0.13      0.15      0.14       144
          13       0.18      0.18      0.18       154
          14       0.07      0.07      0.07       153
          15       0.07      0.03      0.05       151
          16       0.09      0.11      0.10       157
          17       0.11      0.13      0.12       147
          18       0.58    

In [44]:
print(y_pred)
print(y_test_reshaped)

[[4.68535075e-18 9.58688106e-05 2.79025926e-06 1.25076156e-03
  2.33597038e-05 9.89856243e-01 5.38902183e-04 9.33874981e-05
  3.51306517e-04 2.44923052e-04 2.22030957e-03 6.44183252e-04
  1.69263699e-03 1.01384567e-03 1.17066320e-05 3.17437400e-04
  4.35826310e-04 2.62182846e-04 1.15579656e-04 5.47922740e-04
  2.80846405e-04]]
[ 3 10 10 ... 17  3  2]


In [1]:
len(y_pred)

NameError: name 'y_pred' is not defined